In [1]:
import numpy as np
import pandas as pd
import pickle
import sys
import os
import dsc
from dsc.query_engine import Query_Processor as dscQP
from dsc import dsc_io

import matplotlib.pyplot as plt
from pymir import mpl_stylesheet
from pymir import mpl_utils
mpl_stylesheet.banskt_presentation(splinecolor = 'black', dpi = 300)

from mrashpen.inference.mrash_wrapR import MrASHR

import utils

In [2]:
def stratify_dfcol(df, colname, value):
    #return pd_utils.select_dfrows(df, [f"$({colname}) == {value}"])
    return df.loc[df[colname] == value]

def stratify_dfcols(df, condition_list):
    for (colname, value) in condition_list:
        df = stratify_dfcol(df, colname, value)
    return df

def stratify_dfcols_in_list(df, colname, values):
    return df.loc[df[colname].isin(values)]

In [3]:
dsc_output = "/home/saikatbanerjee/scratch/work/gradvi-experiments/linreg_corr_init"
dsc_fname  = os.path.basename(os.path.normpath(dsc_output))
db = os.path.join(dsc_output, dsc_fname + ".db")
elbopkl   = os.path.join("../dsc/results", dsc_fname + "_elbo.pkl")
msepkl    = os.path.join("../dsc/results", dsc_fname + "_mse.pkl")
dscoutpkl = os.path.join("../dsc/results", dsc_fname + "_dscout.pkl")
dscout    = pd.read_pickle(dscoutpkl)

refresh_pickle = False

print(db)
plotprefix = "mrash_elbo_update_after_gradvi_compound"

/home/saikatbanerjee/scratch/work/gradvi-experiments/linreg_corr_init/linreg_corr_init.db


In [4]:
target = ["simulate", "simulate.sfix", "simulate.pve", "simulate.se", "simulate.dims", "fit"]
#condition = ["simulate.sfix == 2", "simulate.signal == 'normal'", "simulate.dims == '(50, 200)'"]
#groups = ["fit_cpt:"]
condition = [""]

In [5]:
qp = dscQP(db, target, condition)
df = qp.output_table
df

,DSC,simulate,simulate.sfix,simulate.se:output,simulate.pve,simulate.output.file,simulate.dims,fit,fit.output.file
0,1,blockdiag,2,blockdiag/blockdiag_1,0.4,blockdiag/blockdiag_1,"(500,10000)",mr_ash,mr_ash/blockdiag_1_mr_ash_1
1,1,blockdiag,5,blockdiag/blockdiag_2,0.4,blockdiag/blockdiag_2,"(500,10000)",mr_ash,mr_ash/blockdiag_2_mr_ash_1
2,1,blockdiag,10,blockdiag/blockdiag_3,0.4,blockdiag/blockdiag_3,"(500,10000)",mr_ash,mr_ash/blockdiag_3_mr_ash_1
3,1,blockdiag,20,blockdiag/blockdiag_4,0.4,blockdiag/blockdiag_4,"(500,10000)",mr_ash,mr_ash/blockdiag_4_mr_ash_1
4,1,blockdiag,2,blockdiag/blockdiag_5,0.6,blockdiag/blockdiag_5,"(500,10000)",mr_ash,mr_ash/blockdiag_5_mr_ash_1
...,...,...,...,...,...,...,...,...,...
715,10,blockdiag,20,blockdiag/blockdiag_116,0.6,blockdiag/blockdiag_116,"(500,10000)",gradvi_compound_lasso_init,gradvi_compound_lasso_init/blockdiag_116_lasso...
716,10,blockdiag,2,blockdiag/blockdiag_117,0.8,blockdiag/blockdiag_117,"(500,10000)",gradvi_compound_lasso_init,gradvi_compound_lasso_init/blockdiag_117_lasso...
717,10,blockdiag,5,blockdiag/blockdiag_118,0.8,blockdiag/blockdiag_118,"(500,10000)",gradvi_compound_lasso_init,gradvi_compound_lasso_init/blockdiag_118_lasso...
718,10,blockdiag,10,blockdiag/blockdiag_119,0.8,blockdiag/blockdiag_119,"(500,10000)",gradvi_compound_lasso_init,gradvi_compound_lasso_init/blockdiag_119_lasso...


In [32]:
if refresh_pickle:
    elbodf = get_elbodf(df)
    elbodf.to_pickle(elbopkl)
else:
    elbodf = pd.read_pickle(elbopkl)

In [33]:
elbodf

,simulate.sfix,simulate.pve,DSC,mr_ash,mr_ash_lasso_init,gradvi_direct,gradvi_compound,gradvi_direct_lasso_init,gradvi_compound_lasso_init,mr_ash_status,mr_ash_lasso_init_status,gradvi_direct_status,gradvi_compound_status,gradvi_direct_lasso_init_status,gradvi_compound_lasso_init_status,gradvi_compound_mrash
0,2,0.4,1,45.118440,25.046803,52.889707,24.514484,25.259207,25.044529,1,1,1,1,1,1,24.2796
1,2,0.4,2,1141.450048,1119.165569,1154.852307,1119.576781,1124.250888,1126.887124,1,1,3,1,1,1,NaN
2,2,0.4,3,612.137333,612.137472,641.928341,596.558639,604.612715,602.086324,1,1,1,1,1,1,NaN
3,2,0.4,4,1192.488452,1174.874965,1193.194599,1174.849111,1181.579979,1181.492859,1,1,1,1,1,1,NaN
4,2,0.4,5,910.915673,892.957743,899.801459,892.941383,912.282278,906.877609,1,1,1,1,1,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,20,0.8,6,1366.084786,1364.682093,1366.355746,1318.238886,1340.154542,1340.421997,1,1,1,1,1,1,NaN
116,20,0.8,7,1229.688501,1231.117763,1230.535359,1200.772301,1217.957233,1212.034480,1,1,1,1,1,1,NaN
117,20,0.8,8,1050.558368,975.072913,1008.139606,983.775283,1021.347193,1000.447799,1,1,1,1,1,1,NaN
118,20,0.8,9,1189.282003,1127.045803,1193.005415,1142.771955,1144.443581,1144.338474,1,1,1,1,1,1,NaN


In [8]:
def read_result(dfrow):
    
    def read_dsc_pickle(fprefix):
        fname = os.path.join(dsc_output, f"{fprefix}.pkl")
        return dsc_io.load_dsc(fname)
    
    # Input data
    data_fprefix = dfrow['simulate.output.file']
    gv_fprefix   = dfrow['fit.output.file']
    data = read_dsc_pickle(data_fprefix)
    gv   = read_dsc_pickle(gv_fprefix)
    return data, gv

def get_mse_from_dscout(df, sfix, pve, dsc, method):
    dfrows = stratify_dfcols(df, 
                             [("simulate.sfix", sfix), 
                              ("simulate.pve", pve), 
                              ("DSC", dsc), 
                              ("fit", method)])
    mse = dfrows[~dfrows['mse.err'].isnull()]['mse.err'].values
    if len(mse) == 1:
        return mse[0]
    else:
        print ("Error fetching value")
        return mse[0]

def get_msedf(elbodf, dscdf):
    methods = [x[:-7] for x in list(elbodf.columns) if x.endswith("_status")]
    msedict = elbodf.to_dict()
    ikeys = list(msedict['simulate.sfix'].keys())
    for method in methods:
        for i in ikeys:
            mse = get_mse_from_dscout(dscout, 
                          msedict['simulate.sfix'][i], 
                          msedict['simulate.pve'][i],
                          msedict['DSC'][i], 
                          method)
            msedict[method][i] = mse
    return pd.DataFrame.from_dict(msedict)

In [9]:
msedf = get_msedf(elbodf, dscout)
msedf

,simulate.sfix,simulate.pve,DSC,mr_ash,mr_ash_lasso_init,gradvi_direct,gradvi_compound,gradvi_direct_lasso_init,gradvi_compound_lasso_init,mr_ash_status,mr_ash_lasso_init_status,gradvi_direct_status,gradvi_compound_status,gradvi_direct_lasso_init_status,gradvi_compound_lasso_init_status
0,2,0.4,1,0.069623,0.064273,0.067863,0.064522,0.064463,0.064530,1,1,1,1,1,1
1,2,0.4,2,4.756549,4.371758,4.580207,4.385927,4.389183,4.410747,1,1,3,1,1,1
2,2,0.4,3,0.615273,0.615078,0.609993,0.608453,0.607220,0.606383,1,1,1,1,1,1
3,2,0.4,4,5.859945,6.170289,5.859629,6.172796,5.877621,5.880089,1,1,1,1,1,1
4,2,0.4,5,2.163899,2.032441,2.032446,2.032304,2.087277,2.070772,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,20,0.8,6,10.755972,10.152884,10.756965,10.372830,10.258590,10.235074,1,1,1,1,1,1
116,20,0.8,7,6.168268,4.771860,6.167810,6.763859,4.920064,4.890841,1,1,1,1,1,1
117,20,0.8,8,3.020773,2.338942,2.838199,2.787267,2.465278,2.426767,1,1,1,1,1,1
118,20,0.8,9,6.605616,5.180304,6.326592,6.062046,5.068147,5.065667,1,1,1,1,1,1


In [28]:
msedict = msedf.to_dict()
elbodict = elbodf.to_dict()
msedict['gradvi_compound_mrash'] = dict()
elbodict['gradvi_compound_mrash'] = dict()

def get_dict_index(dict1, dfrow):
    sfixidx = [i for i, val in dict1['simulate.sfix'].items() if val == dfrow['simulate.sfix']]
    pveidx  = [i for i, val in dict1['simulate.pve'].items() if val == dfrow['simulate.pve']]
    dscidx  = [i for i, val in dict1['DSC'].items() if val == dfrow['DSC']]
    tgtidx  = list(set(dscidx).intersection(set(sfixidx).intersection(set(pveidx))))[0]
    return tgtidx

def get_elbo_mse(dfrow):
    data, gv = read_result(dfrow)
    cavi  = MrASHR(option = "rds", debug = False)
    sk    = gv['model']['prior']['sk']
    winit = gv['model']['prior']['w']
    cavi.fit(data['X'], data['y'], sk, winit = winit, binit = gv['beta_est'], maxiter = 500)
    elbo  = cavi.elbo_path[-1]
    mse   = np.mean(np.square(data['ytest'] - (np.dot(data['Xtest'], cavi.coef[:, 0]) + cavi.fitobj['intercept'])))
    return elbo, mse

In [29]:
gvdf = stratify_dfcol(df, 'fit', 'gradvi_compound')

for idx in range(3):
    dfrow = gvdf.iloc[idx]
    elbo, mse = get_elbo_mse(dfrow)
    elboidx = get_dict_index(elbodict, dfrow)
    print("ELBO index", elboidx)
    elbodict['gradvi_compound_mrash'][elboidx] = elbo
    mseidx  = get_dict_index(msedict, dfrow)
    print("MSE index", mseidx)
    msedict['gradvi_compound_mrash'][mseidx] = mse

2022-11-02 13:14:46,877 | mrashpen.inference.mrash_wrapR | INFO | Mr.ASH terminated at iteration 11.



INFO: Mr.ASH terminated at iteration 11.



ELBO index 0
MSE index 0
2022-11-02 13:14:59,930 | mrashpen.inference.mrash_wrapR | INFO | Mr.ASH terminated at iteration 34.



INFO: Mr.ASH terminated at iteration 34.



ELBO index 10
MSE index 10
2022-11-02 13:15:15,849 | mrashpen.inference.mrash_wrapR | INFO | Mr.ASH terminated at iteration 10.



INFO: Mr.ASH terminated at iteration 10.



ELBO index 20
MSE index 20


In [30]:
new_msedf = pd.DataFrame.from_dict(msedict)
new_elbodf = pd.DataFrame.from_dict(elbodict)
new_elbodf.to_pickle(elbopkl)
new_msedf.to_pickle(msepkl)

,simulate.sfix,simulate.pve,DSC,mr_ash,mr_ash_lasso_init,gradvi_direct,gradvi_compound,gradvi_direct_lasso_init,gradvi_compound_lasso_init,mr_ash_status,mr_ash_lasso_init_status,gradvi_direct_status,gradvi_compound_status,gradvi_direct_lasso_init_status,gradvi_compound_lasso_init_status,gradvi_compound_mrash
0,2,0.4,1,0.069623,0.064273,0.067863,0.064522,0.064463,0.064530,1,1,1,1,1,1,0.064529
1,2,0.4,2,4.756549,4.371758,4.580207,4.385927,4.389183,4.410747,1,1,3,1,1,1,NaN
2,2,0.4,3,0.615273,0.615078,0.609993,0.608453,0.607220,0.606383,1,1,1,1,1,1,NaN
3,2,0.4,4,5.859945,6.170289,5.859629,6.172796,5.877621,5.880089,1,1,1,1,1,1,NaN
4,2,0.4,5,2.163899,2.032441,2.032446,2.032304,2.087277,2.070772,1,1,1,1,1,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,20,0.8,6,10.755972,10.152884,10.756965,10.372830,10.258590,10.235074,1,1,1,1,1,1,NaN
116,20,0.8,7,6.168268,4.771860,6.167810,6.763859,4.920064,4.890841,1,1,1,1,1,1,NaN
117,20,0.8,8,3.020773,2.338942,2.838199,2.787267,2.465278,2.426767,1,1,1,1,1,1,NaN
118,20,0.8,9,6.605616,5.180304,6.326592,6.062046,5.068147,5.065667,1,1,1,1,1,1,NaN


In [31]:
gvdf.shape

(120, 9)